In [1]:
%load_ext autoreload
%autoreload 2

import sympy as s
import numpy as np

In [4]:
gamma = s.Symbol("gamma")
omega = s.Symbol("omega") #freq harmonic oscillator
eta = s.Symbol("eta") #efficiency measurement (appearing in the CPTP decomposiion X,Y)
kappa = s.Symbol("kappa") #efficiency measurement (appearing in the CPTP decomposiion X,Y)
n = s.Symbol("n") #n_th

Omega = s.Matrix(np.array([[0,1],[-1,0]])) ##Simplectic matrix

In [5]:
Cint = -s.sqrt(gamma)*Omega # -\ii \Sigma_y

aint = Omega*Cint*Omega*(Cint.T)/2
Hs = s.eye(2)*omega
asys = Omega*Hs
A_matrix = aint + asys ##drift matrix
#D_matrix = Omega*Cint*sigma_b*(Omega*Cint).T  #difussion matrix
D_matrix = s.Matrix(np.diag([gamma*(n + 0.5)]*2))
Gamma_matrix = s.Matrix(np.zeros((2,2)))
C_matrix = s.Matrix(np.array([[s.sqrt(2*eta*kappa),0],[0,0]]))

In [8]:
varx, varp, covxp = [s.Symbol(k) for k in ["varx","varp","covxp"]]
cov = s.Matrix([[varx, covxp],[covxp, varp]])

ChiCov = cov*C_matrix.T + Gamma_matrix.T

In [9]:
dots = np.array(A_matrix*cov + cov*A_matrix.T + D_matrix - ChiCov.T*ChiCov)
ders = dots[0,0], dots[0,1], dots[1,1]
[ders]

[(-2*covxp**2*eta*kappa + 2*covxp*omega - gamma*varx + gamma*(n + 0.5) - 2*varx**2*eta*kappa,
  -covxp*gamma + omega*varp - omega*varx,
  -2*covxp*omega - gamma*varp + gamma*(n + 0.5))]

In [6]:
## interaction hamiltonian between the train of modes being measured and system

Cint = -s.sqrt(gamma)*Omega # -\ii \Sigma_y
s.Matrix(Omega)*(s.Matrix(Cint))
Hint = np.block([[s.zeros(2), Cint], [Cint.T, s.zeros(2)]])/2

xs, ps, xb, pb = s.Symbol("xs"), s.Symbol("ps"), s.Symbol("xb"), s.Symbol("pb")

print((s.Matrix([xs,ps,xb,pb]).T)*Hint*s.Matrix([xs,ps,xb,pb])) #BS splitter
aint = Omega*Cint*Omega*(Cint.T)/2
Hs = s.eye(2)*omega
asys = Omega*Hs


Matrix([[-sqrt(gamma)*pb*xs + sqrt(gamma)*ps*xb]])


In [7]:
#noisy heterodyne measurement
S = s.eye(2) 
sigma_m = S*(S.T)
##lossy part
Xstar = s.eye(2)/s.sqrt(eta)
Ystar = s.eye(2)*(1-eta)/eta
sigma_noisy_m = Xstar*(sigma_m)*(Xstar.T) + Ystar

# state of B, subsystem being measured
sigma_b = (2*N +1)*s.eye(2)

Gamma_matrix = Omega*Cint*sigma_b   #auxiliar matrix for the ricatti

C_matrix = (Cint*Omega*(sigma_b + sigma_noisy_m)**(-1/2)).T

 Ricatti equation for covariance

In [13]:
A_matrix

Matrix([
[-gamma/2,    omega],
[  -omega, -gamma/2]])

In [14]:
from scipy.linalg import solve_continuous_are
A_tilde_matrix = A_matrix - (Omega*Cint*sigma_b*((sigma_b + sigma_noisy_m)**(-1))*Omega*Cint.T   )
D_tilde_matrix = D_matrix + (Omega*Cint*sigma_b*((sigma_b + sigma_noisy_m)**(-1))*sigma_b*Cint.T*Omega)
B_matrix = Cint*Omega*(sigma_b + sigma_noisy_m)**(-1/2)

In [15]:
Ar, Br, Dr = [np.array(k.subs({omega:2*np.pi, N:2, eta:0.8})).astype(np.float32) for k in [A_tilde_matrix, B_matrix, D_tilde_matrix]]

TypeError: can't convert expression to float

In [164]:
cov_stat = solve_continuous_are(Ar, Br, Dr, np.eye(2))

In [166]:
varx, varp, covxp = [s.Symbol(k) for k in ["varx","varp","covxp"]]
cov = s.Matrix([[varx, covxp],[covxp, varp]])

In [191]:
varx, varp, covxp = [s.Symbol(k) for k in ["varx","varp","covxp"]]
cov = s.Matrix([[varx, covxp],[covxp, varp]])

dots = np.array(A_tilde_matrix*cov + cov*A_tilde_matrix.T + D_tilde_matrix - B_matrix*B_matrix.T)
ders = dots[0,0], dots[0,1], dots[1,1]
[ders]

[(2*N + 2*covxp*omega + 2*varx*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) - (2*N + 1)**2/(2*N + 1 + (1 - eta)/eta + 1/eta) - (2*N + 1 + (1 - eta)/eta + 1/eta)**(-1.0) + 1,
  2*covxp*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) + omega*varp - omega*varx,
  2*N - 2*covxp*omega + 2*varp*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) - (2*N + 1)**2/(2*N + 1 + (1 - eta)/eta + 1/eta) - (2*N + 1 + (1 - eta)/eta + 1/eta)**(-1.0) + 1)]

In [49]:
a = ders[0]

'-2*covxp**2*eta*kappa + 2*covxp*omega - gamma*varx + gamma*(n + 0.5) - 2*varx**2*eta*kappa'

In [ ]:
def Euler(ff, G, y0, tspan, dt,**kwargs):
    exp = kwargs.get("exp",False)
    N = len(tspan)
    y = np.zeros((N, len(y0)))
    y[0] = y0

    if exp is True:
        dint = 1.
    else:
        dint= dt

    for ind,t in enumerate(tqdm(tspan[:-1])):
        #dW = np.random.normal(0,np.sqrt(dt), (7))
        w0 = np.random.normal()*np.sqrt(dt)
        w1 = np.random.normal()*np.sqrt(dt)
        dW = np.array([w0, w1, w0, w1 , 0.,0.,0.])
        y[ind+1] = y[ind] + ff(y[ind], t, exp=exp, dt=dt)*dint + np.dot(G(y[ind], t), dW)
    return y

In [194]:
from integrate import *

In [202]:
x0 = 1.
p0 = 0.
yx0 = 0.
yp0 = 0.
varx0 = 1.
varp0 = 1.
covxy0 = 0.
s0 = np.array([x0, p0, yx0, yp0, varx0, varp0,covxy0])

gamma = 0.3
N = 2.0
eta = 0.8

periods = 10
ppp=1000
dt = 1/ppp
times = np.arange(0.,periods+dt,dt)

global C, A , D 
C = C_matrix
D = D_matrix
A = A_matrix


coeffs = [C, A, D , dt]
params = [eta, gamma, omega, N]

solution = RosslerSRI2(Fs, Gs, s0, times, dt)

  0%|          | 0/10000 [00:00<?, ?it/s]


NameError: name 'A' is not defined